In [1]:
import os
import re
import time
from pathlib import Path
import shutil
import glob
import string
import unicodedata
from datetime import datetime, timedelta

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from sqlalchemy import create_engine
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from unicodedata import normalize, category

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stopset = set(stopwords.words("spanish"))

In [3]:
print(stopset)

{'algo', 'sentid', 'tenida', 'fuiste', 'tenías', 'sentidos', 'tuvieseis', 'seríamos', 'están', 'tenidas', 'han', 'hubieran', 'antes', 'tuyo', 'mucho', 'tenéis', 'hubiésemos', 'estad', 'lo', 'me', 'hemos', 'hubiera', 'serían', 'también', 'tenga', 'sentidas', 'estéis', 'los', 'serías', 'cual', 'está', 'estás', 'estuvieses', 'otro', 'fue', 'fueses', 'estuvierais', 'nuestro', 'nos', 'tú', 'fuéramos', 'tendrán', 'suya', 'fuésemos', 'estaríais', 'tendremos', 'esas', 'estoy', 'eras', 'nosotros', 'tuviera', 'estuvieron', 'habríamos', 'hay', 'estén', 'era', 'tuve', 'habrás', 'estamos', 'tienes', 'sentida', 'habiendo', 'vuestros', 'estaría', 'os', 'tuvimos', 'ya', 'habrían', 'estando', 'las', 'tuvierais', 'le', 'pero', 'habíamos', 'esta', 'suyos', 'tuya', 'habida', 'tuvieses', 'estará', 'habíais', 'estabas', 'estaremos', 'sin', 'seremos', 'no', 'se', 'estaréis', 'cuando', 'tenemos', 'has', 'eres', 'tengan', 'más', 'otra', 'nuestras', 'siente', 'hubieron', 'estos', 'sintiendo', 'habrías', 'te', '

In [4]:
hoy = datetime.today()
hoy_format = hoy.strftime('%d%m%Y') 
hoy_format

'12092021'

In [5]:
for file in glob.iglob('news/{}/**/*.txt'.format(hoy_format), recursive=True):
    listado = file.replace('/', '\\').split('\\')
    
    if 'descartado' in listado:
        continue
        
    print(listado)
    with open(file, encoding='UTF-8') as f:
        new = f.read()
        
    break

['news', '12092021', 'original', 'bono-350-quienes-podran-cobrarlo-desde-el-13-de-septiembre-y-de-que-forma-bono-yanapay-peru-coronavirus-covi.txt']


In [6]:
from unicodedata import normalize, category

def clean_text_1(_text):
    """
        Ll : minusculas
        Zs : espacios
        Lu : mayusculas
        Nd : numeros
        Mn : tildes
        Ps : abre dieresis, .....
        Pe : cierra dieresis, .......
        Pi : abre parentesis, .....
        Po : cierra parentesis, punto, coma, .....        
        ...
    """
    selects_characters = ['Ll', 'Zs', 'Lu']
    
    return ''.join([_.lower() for _ in normalize('NFD', _text.replace(' ', 'ZZZ')) if category(_) in selects_characters])

### SIN QUITAR STOPWORDS

In [7]:
tokenizacion = [
    [clean_text_1(_) for _ in nltk.word_tokenize(sentence) if len(clean_text_1(_)) > 0] for sentence in nltk.sent_tokenize(new)
]

for oracion in tokenizacion:
    print(oracion)
    print("-"*100)

['desde', 'la', 'tercera', 'semana', 'de', 'setiembre', 'se', 'empezara', 'a', 'entregar', 'el', 'bono', 'soles']
----------------------------------------------------------------------------------------------------
['este', 'subsidio', 'tambien', 'conocido', 'como', 'yanapay', 'peru', 'tiene', 'como', 'objetivo', 'ser', 'una', 'ayuda', 'economica', 'para', 'aliviar', 'a', 'los', 'sectores', 'mas', 'vulnerables', 'ante', 'la', 'pandemia', 'de', 'coronavirus']
----------------------------------------------------------------------------------------------------
['este', 'bono', 'es', 'una', 'inyeccion', 'economica', 'universal', 'individualizada', 'que', 'a', 'diferencia', 'de', 'otros', 'subsidios', 'que', 'el', 'estado', 'ya', 'ha', 'venido', 'entregando', 'tiene', 'como', 'principal', 'caracteristica', 'que', 'sera', 'pagado', 'de', 'forma', 'personal', 'y', 'no', 'familiar']
----------------------------------------------------------------------------------------------------
['con', 'es

### QUITANDO STOPWORDS

In [8]:
stopset_cleaned = [clean_text_1(word) for word in stopset]
print(stopset_cleaned)

['algo', 'sentid', 'tenida', 'fuiste', 'tenias', 'sentidos', 'tuvieseis', 'seriamos', 'estan', 'tenidas', 'han', 'hubieran', 'antes', 'tuyo', 'mucho', 'teneis', 'hubiesemos', 'estad', 'lo', 'me', 'hemos', 'hubiera', 'serian', 'tambien', 'tenga', 'sentidas', 'esteis', 'los', 'serias', 'cual', 'esta', 'estas', 'estuvieses', 'otro', 'fue', 'fueses', 'estuvierais', 'nuestro', 'nos', 'tu', 'fueramos', 'tendran', 'suya', 'fuesemos', 'estariais', 'tendremos', 'esas', 'estoy', 'eras', 'nosotros', 'tuviera', 'estuvieron', 'habriamos', 'hay', 'esten', 'era', 'tuve', 'habras', 'estamos', 'tienes', 'sentida', 'habiendo', 'vuestros', 'estaria', 'os', 'tuvimos', 'ya', 'habrian', 'estando', 'las', 'tuvierais', 'le', 'pero', 'habiamos', 'esta', 'suyos', 'tuya', 'habida', 'tuvieses', 'estara', 'habiais', 'estabas', 'estaremos', 'sin', 'seremos', 'no', 'se', 'estareis', 'cuando', 'tenemos', 'has', 'eres', 'tengan', 'mas', 'otra', 'nuestras', 'siente', 'hubieron', 'estos', 'sintiendo', 'habrias', 'te', '

In [9]:
tokenizacion = [
    [
        clean_text_1(_) for _ in nltk.word_tokenize(sentence) 
                     if len(clean_text_1(_)) > 0 and clean_text_1(_) not in stopset_cleaned
    ] for sentence in nltk.sent_tokenize(new)
]

for oracion in tokenizacion:
    if len(oracion) <= 0:
        continue

    print(oracion)
    print("-"*100)

['tercera', 'semana', 'setiembre', 'empezara', 'entregar', 'bono', 'soles']
----------------------------------------------------------------------------------------------------
['subsidio', 'conocido', 'yanapay', 'peru', 'objetivo', 'ser', 'ayuda', 'economica', 'aliviar', 'sectores', 'vulnerables', 'pandemia', 'coronavirus']
----------------------------------------------------------------------------------------------------
['bono', 'inyeccion', 'economica', 'universal', 'individualizada', 'diferencia', 'subsidios', 'venido', 'entregando', 'principal', 'caracteristica', 'pagado', 'forma', 'personal', 'familiar']
----------------------------------------------------------------------------------------------------
['subsidio', 'millones', 'personas', 'beneficiadas', 'peru']
----------------------------------------------------------------------------------------------------
['bono', 'otorgara', 'personas', 'mayores', 'edad', 'cuyos', 'ingresos', 'superen', 's', 'mensuales', 'formen', 'part

### LEMMATIZACION

In [10]:
#!pip install spacy                                  -- descomentar la 1era vez

In [11]:
#!pip install stanza                                 -- descomentar la 1era vez

In [12]:
#!pip install "spacy-stanza<0.3.0" --user            -- descomentar la 1era vez

In [13]:
import stanza

In [14]:
#stanza.download('es')                                -- descomentar la 1era vez

In [15]:
from spacy_stanza import StanzaLanguage

In [16]:
snlp = stanza.Pipeline(lang="es")
nlp = StanzaLanguage(snlp)

2021-09-12 10:56:50 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-09-12 10:56:50 INFO: Use device: cpu
2021-09-12 10:56:50 INFO: Loading: tokenize
2021-09-12 10:56:50 INFO: Loading: mwt
2021-09-12 10:56:50 INFO: Loading: pos
2021-09-12 10:56:51 INFO: Loading: lemma
2021-09-12 10:56:51 INFO: Loading: depparse
2021-09-12 10:56:51 INFO: Loading: ner
2021-09-12 10:56:54 INFO: Done loading processors!


In [17]:
tokenizacion = [
    [
       ''.join([token.lemma_ for token in nlp(clean_text_1(_))]) for _ in nltk.word_tokenize(sentence) 
                     if len(clean_text_1(_)) > 0 and clean_text_1(_) not in stopset_cleaned
    ] for sentence in nltk.sent_tokenize(new)
]

for oracion in tokenizacion:
    if len(oracion) <= 0:
        continue

    print(oracion)
    print("-"*100)

['tercero', 'semana', 'setiembre', 'empezar', 'entregar', 'bono', 'sol']
----------------------------------------------------------------------------------------------------
['subsidio', 'conocido', 'yanapar', 'peru', 'objetivo', 'ser', 'ayuda', 'economico', 'aliviar', 'sector', 'vulnerable', 'pandemia', 'coronaviru']
----------------------------------------------------------------------------------------------------
['bono', 'inyeccion', 'economico', 'universal', 'individualizado', 'diferencia', 'subsidio', 'venir', 'entregar', 'principal', 'caracteristica', 'pagado', 'forma', 'personal', 'familiar']
----------------------------------------------------------------------------------------------------
['subsidio', 'millón', 'persona', 'beneficiado', 'peru']
----------------------------------------------------------------------------------------------------
['bono', 'otorgar', 'persona', 'mayor', 'edad', 'cuyo', 'ingreso', 'superar', 's', 'mensual', 'formar', 'parte', 'hogar', 'condicion

### REGRESANDO A MODO TEXTO

In [19]:
textos = [' '.join(sentence) for sentence in tokenizacion]
print(textos)

['tercero semana setiembre empezar entregar bono sol', 'subsidio conocido yanapar peru objetivo ser ayuda economico aliviar sector vulnerable pandemia coronaviru', 'bono inyeccion economico universal individualizado diferencia subsidio venir entregar principal caracteristica pagado forma personal familiar', 'subsidio millón persona beneficiado peru', 'bono otorgar persona mayor edad cuyo ingreso superar s mensual formar parte hogar condicion pobreza pobreza extremo vulnerabilidad pais', 'recibir beneficiario programa junto pension tú', 'ser beneficiario bono ser mayor edad pertenecer alguno grupo cuatro grupo beneficiario cada fecha partir poder cobrar bono', 'determinado cronograma pago segun grupo caber resaltar persona pertenecer grupo poder recibir bono mediante parte persona grupo recibir subsidio trave atencion ventanilla banco nacion', 'tras conocerél grupo fecha bono entregado gobierno dado conocer modalidad pago pagar', 'caber aclarar ajustar grupo ciudadano encontrar', 'aun p

### BOLSA DE PALABRAS

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv_fit = cv.fit_transform(textos)

In [22]:
cv_fit

<14x119 sparse matrix of type '<class 'numpy.int64'>'
	with 162 stored elements in Compressed Sparse Row format>

In [24]:
print([name for name in dir(cv_fit) if '__' not in name])

['_add_dense', '_add_sparse', '_arg_min_or_max', '_arg_min_or_max_axis', '_asindices', '_binopt', '_deduped_data', '_divide', '_divide_sparse', '_get_arrayXarray', '_get_arrayXint', '_get_arrayXslice', '_get_columnXarray', '_get_dtype', '_get_intXarray', '_get_intXint', '_get_intXslice', '_get_sliceXarray', '_get_sliceXint', '_get_sliceXslice', '_get_submatrix', '_imag', '_inequality', '_insert_many', '_major_index_fancy', '_major_slice', '_maximum_minimum', '_min_or_max', '_min_or_max_axis', '_minor_index_fancy', '_minor_reduce', '_minor_slice', '_mul_multivector', '_mul_scalar', '_mul_sparse_matrix', '_mul_vector', '_prepare_indices', '_process_toarray_args', '_real', '_rsub_dense', '_scalar_binopt', '_set_arrayXarray', '_set_arrayXarray_sparse', '_set_dtype', '_set_intXint', '_set_many', '_set_self', '_setdiag', '_shape', '_sub_dense', '_sub_sparse', '_swap', '_validate_indices', '_with_data', '_zero_many', 'arcsin', 'arcsinh', 'arctan', 'arctanh', 'argmax', 'argmin', 'asformat', 'a

In [27]:
print(cv.get_feature_names())

['abril', 'acceder', 'aclarar', 'ajustar', 'alguno', 'aliviar', 'ano', 'apoyo', 'arca', 'atencion', 'aun', 'ayuda', 'banco', 'beneficiado', 'beneficiario', 'bono', 'caber', 'cada', 'caracteristica', 'ciudadano', 'cobrar', 'condicion', 'conocer', 'conocerél', 'conocido', 'consulta', 'consultar', 'coronaviru', 'cronograma', 'cuatro', 'cuyo', 'dado', 'dar', 'deber', 'despues', 'determinado', 'dia', 'diferencia', 'dinero', 'direccion', 'economico', 'edad', 'empezar', 'encontrar', 'enlace', 'entidad', 'entregado', 'entregar', 'extremo', 'familiar', 'fecha', 'financiero', 'forma', 'formar', 'gobierno', 'grupo', 'hacer', 'hogar', 'individual', 'individualizado', 'ingreso', 'inyeccion', 'junto', 'link', 'mayor', 'mediante', 'mensual', 'millón', 'modalidad', 'monetario', 'nacion', 'objetivo', 'otorgar', 'pagado', 'pagar', 'pago', 'pais', 'pandemia', 'parte', 'partir', 'pendiente', 'pension', 'persona', 'personal', 'pertenecer', 'peru', 'pobreza', 'poder', 'portal', 'principal', 'programa', 'pro

In [30]:
print(cv.vocabulary_)

{'tercero': 108, 'semana': 101, 'setiembre': 103, 'empezar': 42, 'entregar': 47, 'bono': 15, 'sol': 105, 'subsidio': 106, 'conocido': 24, 'yanapar': 118, 'peru': 85, 'objetivo': 71, 'ser': 102, 'ayuda': 11, 'economico': 40, 'aliviar': 5, 'sector': 99, 'vulnerable': 116, 'pandemia': 77, 'coronaviru': 27, 'inyeccion': 61, 'universal': 112, 'individualizado': 59, 'diferencia': 37, 'venir': 113, 'principal': 89, 'caracteristica': 18, 'pagado': 73, 'forma': 52, 'personal': 83, 'familiar': 49, 'millón': 67, 'persona': 82, 'beneficiado': 13, 'otorgar': 72, 'mayor': 64, 'edad': 41, 'cuyo': 30, 'ingreso': 60, 'superar': 107, 'mensual': 66, 'formar': 53, 'parte': 78, 'hogar': 57, 'condicion': 21, 'pobreza': 86, 'extremo': 48, 'vulnerabilidad': 115, 'pais': 76, 'recibir': 93, 'beneficiario': 14, 'programa': 90, 'junto': 62, 'pension': 81, 'tú': 111, 'pertenecer': 84, 'alguno': 4, 'grupo': 55, 'cuatro': 29, 'cada': 17, 'fecha': 50, 'partir': 79, 'poder': 87, 'cobrar': 20, 'determinado': 35, 'crono

In [33]:
tamanio_vocabulario = len(cv.vocabulary_)
tamanio_vocabulario

119

In [34]:
cantidad_textos = len(textos)
cantidad_textos

14

In [36]:
espacios_con_ceros = cv_fit.toarray().sum(axis=1).sum()
espacios_con_ceros

172

### Porcentaje de ceros

In [38]:
sparsity = 1 - 172/ (119*14)
sparsity

0.8967587034813925